In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import confidence.confidence_tools
import py3Dmol
import subprocess


In [22]:
abspath = os.getcwd()
abspath = abspath.split("abag-benchmark-set")[0] + "abag-benchmark-set"

In [23]:
"""Top 5 and bottom 5 abag_dockq scores

pdbid  median
97   8vk1    0.93
104  8ysh    0.93
73   8pe1    0.92
38   8c5h    0.92
64   8jeo    0.90
..    ...     ...
13   7t0l    0.01
3    7q0i    0.01
95   8u3s    0.01
43   8e7m    0.00
15   7t82    0.00
"""

'Top 5 and bottom 5 abag_dockq scores\n\npdbid  median\n97   8vk1    0.93\n104  8ysh    0.93\n73   8pe1    0.92\n38   8c5h    0.92\n64   8jeo    0.90\n..    ...     ...\n13   7t0l    0.01\n3    7q0i    0.01\n95   8u3s    0.01\n43   8e7m    0.00\n15   7t82    0.00\n'

In [24]:
id="7t82" # 7ox2complex  7t82complex  8u3scomplex
model=3
prediction=10
preset="alphafold3"

id="8hit"
model=1
prediction=5
preset="alphafold3"

id="8hit"
model=1
prediction=0
preset="alphafold3"

In [25]:
# Software
MMalign_exe="/home/sfromm/programs/MMalign/MMalign"

# Directories
tmp = "/home/sfromm/tmp"
reference = f"/home/sfromm/git/abag-benchmark-set/data/db/structures_filtered/{id}_filtered.pdb"
reference_orig = f"/home/sfromm/git/abag-benchmark-set/data/db/structures/{id}.pdb"

#AF2
path_scores = f"/home/sfromm/git/abag-benchmark-set/data/scores_ae/{preset}/output/{id}/{id}_model_{model}_multimer_v3_pred_{prediction}_{preset}/{id}_model_{model}_multimer_v3_pred_{prediction}_{preset}_merged.csv"
query_cut = f"/home/sfromm/git/abag-benchmark-set/data/scores_ae/{preset}/output/{id}/{id}_model_{model}_multimer_v3_pred_{prediction}_{preset}/{id}_model_{model}_multimer_v3_pred_{prediction}_{preset}_cut_query.pdb"
reference_cut = f"/home/sfromm/git/abag-benchmark-set/data/scores_ae/{preset}/output/{id}/{id}_model_{model}_multimer_v3_pred_{prediction}_{preset}/{id}_model_{model}_multimer_v3_pred_{prediction}_{preset}_cut_reference.pdb"

#AF3
path_scores = f"/home/sfromm/git/abag-benchmark-set/data/scores_ae/{preset}/output/{id}/{id}_seed_{prediction}_sample_{model}_{preset}/{id}_seed_{prediction}_sample_{model}_{preset}_merged.csv"
query_cut = f"/home/sfromm/git/abag-benchmark-set/data/scores_ae/{preset}/output/{id}/{id}_seed_{prediction}_sample_{model}_{preset}/{id}_seed_{prediction}_sample_{model}_{preset}_cut_query.pdb"
reference_cut = f"/home/sfromm/git/abag-benchmark-set/data/scores_ae/{preset}/output/{id}/{id}_seed_{prediction}_sample_{model}_{preset}/{id}_seed_{prediction}_sample_{model}_{preset}_cut_reference.pdb"
query = f"/home/sfromm/git/abag-benchmark-set/data/models/alphafold3/{id}complex/model_seed_{prediction}_sample_{model}_{preset}.pdb"



In [26]:
# Some helper functions
def get_chains(pdb):
    return [chain.id for chain in confidence.confidence_tools.load_pdb_structure(pdb).get_chains()]

def get_name(pdb):
    return os.path.split(pdb)[1].split('.')[0]

# Align target structures to template structure

def MMalign_wrapper(MMalign_exe, target_pdb, template_pdb, aligned_pdb=None):
    if aligned_pdb is None:
        raw_cmd = [MMalign_exe, target_pdb, template_pdb, "-outfmt 2"]
    else:
        raw_cmd = [MMalign_exe, target_pdb, template_pdb, "-outfmt 2", "-o", aligned_pdb]
    cmd = " ".join(raw_cmd)
    result = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, text=True).stdout
    keys = result.split("\n")[0][1:].split("\t")
    values = result.split("\n")[1].split("\t")
    result = {}
    result["target"] =  os.path.split(target_pdb)[1].split('.')[0]
    result["template"] =  os.path.split(template_pdb)[1].split('.')[0]
    result.update({keys[i] : values[i] for i in range(len(keys))})
    result["PDBchain1"] = result["PDBchain1"].split(":",1)[1]
    result["PDBchain2"] = result["PDBchain2"].split(":",1)[1]
    return result


In [27]:


color_list = ['#33ff33', '#00ffff', '#ff33cc', '#ffff00', '#ff9999', '#e5e5e5', '#7f7fff', '#ff7f00', '#7fff7f', '#199999', '#ff007f', '#ffdd5e', '#8c3f99', '#b2b2b2', '#007fff', '#c4b200', '#8cb266', '#00bfbf', '#b27f7f', '#fcd1a5', '#ff7f7f', '#ffbfdd', '#7fffff', '#ffff7f', '#00ff7f', '#337fcc', '#d8337f', '#bfff3f', '#ff7fff', '#d8d8ff', '#3fffbf', '#b78c4c', '#339933', '#66b2b2', '#ba8c84', '#84bf00', '#b24c66', '#7f7f7f', '#3f3fa5', '#a5512b']
#alphabet_list = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
color_list = ['yellow','blue','red','orange','purple','cyan','green','brown','aquamarine','darkblue','lightblue','magenta','pink']

# label options for atom
# atom.atom, atom.chain (chain), atom.resi (residue number), atom.elem (element symbol), atom.resn (name of residue), atom.model (the model the atom belongs to), atom.x, atom.y, atom.z (coordinates)
hover_func = '''function(atom,viewer,event,container) {
                   if(!atom.label) {
                    atom.label = viewer.addLabel(atom.model+":"+atom.chain+":"+atom.resi,{position: atom, backgroundColor: 'mintcream', fontColor:'black'});
                   }}'''
unhover_func = '''function(atom,viewer) { 
                   if(atom.label) {
                    viewer.removeLabel(atom.label);
                    delete atom.label;
                   }
                }'''

def plot_single_structure(view,pdb,pos):
    #color_list = ['#33ff33', '#00ffff', '#ff33cc', '#ffff00', '#ff9999', '#e5e5e5', '#7f7fff', '#ff7f00', '#7fff7f', '#199999', '#ff007f', '#ffdd5e', '#8c3f99', '#b2b2b2', '#007fff', '#c4b200', '#8cb266', '#00bfbf', '#b27f7f', '#fcd1a5', '#ff7f7f', '#ffbfdd', '#7fffff', '#ffff7f', '#00ff7f', '#337fcc', '#d8337f', '#bfff3f', '#ff7fff', '#d8d8ff', '#3fffbf', '#b78c4c', '#339933', '#66b2b2', '#ba8c84', '#84bf00', '#b24c66', '#7f7f7f', '#3f3fa5', '#a5512b']
    # Plot native structure
    view.addModel(open(pdb, 'r').read(),'pdb',viewer=pos)
    chains = get_chains(pdb)
    for i,chain in enumerate(chains):
       view.setStyle({'chain':chain},{'cartoon': {'color':color_list[i]}},viewer=pos)
    return f"Position {pos}:\n\tmodel 0: {get_name(pdb)} ({','.join(repr(item) for item in chains)}) ({','.join(color_list[i] for i,chain in enumerate(chains))})\n"

def plot_multi_structure(view,pdb_lst,pos):
    legend = f"Position {pos}:\n"
    for i, pdb in enumerate(pdb_lst):
        chains = get_chains(pdb)
        view.addModel(open(pdb, 'r').read(),'pdb',viewer=pos)
        view.setStyle({'model':i}, {'cartoon': {'color':color_list[i]}},viewer=pos)
        legend+= f"\tmodel {i}: {get_name(pdb)} ({','.join(repr(item) for item in chains)}) ({color_list[i]})\n"
    return legend


In [28]:

aln_reference_to_reference_orig = os.path.join(tmp,"aln_reference_to_reference_orig.pdb")
aln_query_cut_to_reference = os.path.join(tmp,"aln_query_cut_to_reference.pdb")
aln_reference_cut_to_reference = os.path.join(tmp,"aln_reference_cut_to_reference.pdb")
aln_query_to_reference = os.path.join(tmp,"aln_query_to_reference.pdb")
aln_query_cut_to_reference_cut = os.path.join(tmp,"aln_query_cut_to_reference_cut.pdb")
aln_query_to_reference_orig = os.path.join(tmp,"aln_query_to_reference_orig.pdb")


MMalign_reference_to_reference_orig = MMalign_wrapper(MMalign_exe,reference,reference_orig,aln_reference_to_reference_orig)
MMalign_query_cut_to_reference = MMalign_wrapper(MMalign_exe,query_cut,reference,aln_query_cut_to_reference)
MMalign_reference_cut_to_reference = MMalign_wrapper(MMalign_exe,reference_cut,reference,aln_reference_cut_to_reference)
MMalign_query_to_reference = MMalign_wrapper(MMalign_exe,query,reference,aln_query_to_reference)
MMalign_query_cut_to_reference_cut = MMalign_wrapper(MMalign_exe,query_cut,reference_cut,aln_query_cut_to_reference_cut)
MMalign_aln_query_to_reference_orig = MMalign_wrapper(MMalign_exe,query,reference_orig,aln_query_to_reference_orig)


# Plot aligned structures
view = py3Dmol.view(width=3*400,height=600,viewergrid=(5,2),linked=False)
view.removeAllModels()
view.setBackgroundColor('0x17202a')
legend = plot_single_structure(view,reference_orig,(0,0))
legend = plot_single_structure(view,reference,(0,1))
legend = plot_single_structure(view,query,(1,1))
legend +=plot_multi_structure(view,[aln_reference_to_reference_orig,reference_orig],(2,0))
legend +=plot_multi_structure(view,[aln_query_to_reference,reference],(2,1))
legend +=plot_multi_structure(view,[aln_query_cut_to_reference,reference],(3,0))
legend +=plot_multi_structure(view,[aln_reference_cut_to_reference,reference],(3,1))
legend +=plot_multi_structure(view,[aln_query_cut_to_reference_cut,reference_cut],(4,0))
legend +=plot_multi_structure(view,[aln_query_to_reference_orig,reference_orig],(4,1))
view.setHoverable({},True,hover_func,unhover_func)
view.zoomTo()
print(legend)
view.render()

Warning! Cannot parse file: /home/sfromm/git/abag-benchmark-set/data/models/alphafold3/8hitcomplex/model_seed_0_sample_1_alphafold3.pdb. Chain number 0.


IndexError: list index out of range